<a href="https://colab.research.google.com/github/mattiadg2001/Machine-Learning-Labs/blob/main/Copy_of_ML2025_01_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression

## The Dataset


Let us consider the [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set).

In the dataset we have data regarding specific species of flowers :
- Sepal length;
- Sepal width;
- Petal length;
- Petal width;
- Species (*Iris setosa*, *Iris virginica* e *Iris versicolor*).

In the specific, we have N = 150 total samples (50 per class).

<img src='https://drive.google.com/uc?id=1cBVClKfJOVXwK-VCjwd9XzRgCN-wvec_' width=250>

We need to import **matplotlib** and **pandas** to handle data and plots.

In [ ]:
import pandas as pd #https://pandas.pydata.org/
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

We can find the dataset we need to analyse online. We use pandas to load the csv to a **pandas.DataFrame**.

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = iris_dataset = pd.read_csv(url, names=names)

We can start to have a look the data we have

In [ ]:
dataset.head()

we do not care about the flower species in this lesson, hence we remove that column:

In [ ]:
dataset = dataset.drop('class', axis=1)

In [ ]:
dataset.head()

We will try to understand how the feature are distributed, by printing some statistics:

In [ ]:
dataset.describe()

Visualizing data can also be very helpful:

In [ ]:
dataset.hist(figsize=(16,9))
plt.show()

In [ ]:
scatter_matrix(dataset, figsize=(16, 9))
plt.show()

*petal-lenght* and *petal-width* seem to have a strong relationship... we should investigate it more in detail!

In [ ]:
dataset.plot.scatter('petal-length', 'petal-width', grid=True, figsize=(16,9))
plt.show()

# Predicting Petal Width from Petal Length

### Data preprocessing

Once we inspected the data, we should operate some preprocessing procedures. On a
generic dataset one should perform:

- shuffling;
- remove inconsistent data;
- remove outliers;
- normalize or standardize data;
- fill missing data.

In this case we are going to use the entire dataset, with a non-iterative method, hence we do not need to **shuffle**.

There seems not to be **outliers** from previous inspection.

Is there any **missing data**?

In [ ]:
import numpy as np #https://numpy.org/

In [ ]:
np.any(np.isnan(dataset.values))

we are lucky, no missing data, no outliers...

However it is always better to work with data in the same scale, hence we should normalize the columns we are going to use.

\begin{align*}
	s &\leftarrow \frac{s - \bar{s}}{S} \\
	s &\leftarrow \frac{s - \min_n \{ s_n \}}{\max_n \{ s_n \} - \min_n \{ s_n \}}
\end{align*}

The **zscore** function operates a standardization of its inputs.

In [ ]:
from scipy.stats import zscore #https://scipy.org/

In [ ]:
x = zscore(dataset['petal-length'].values).reshape(-1, 1) # we reshape our feature column as a (n_sample, n_features) matrix
y = zscore(dataset['petal-width'].values)

In [ ]:
x.shape

In [ ]:
np.mean(x)

In [ ]:
np.std(x)

## Using Scikit-Learn Toolbox

A linear model seems to be a good choice to predict *petal-width* given petal-length, let's use **scikit-learn** tools to do a linear regression:


In [ ]:
from sklearn import linear_model #https://scikit-learn.org/stable/

In [ ]:
lin_model = linear_model.LinearRegression()
lin_model.fit(x, y)

In [ ]:
lin_model.coef_

In [ ]:
lin_model.intercept_

since we want to customize our plot, we will use matplotlib directly this time:

In [ ]:
plt.figure(figsize=(16,9))
plt.scatter(x, y, label='true')

w1 = lin_model.coef_ # weights of the model are stored here
w0 = lin_model.intercept_ # and here it is the intercept

# Compute the y component of the regression line

y_pred = lin_model.predict(x)
#y_pred = [w1 * sample + w0 for sample in x.flatten()]

# (we used a list comprehension here, have a look to the python tutorial
#  if you don't know what it is!)

plt.plot(x, y_pred, label='predicted', color='red')

# enlarging fonts
plt.legend(prop={'size': 20})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

## Metrics

To evaluate the quality of our regression we can analyse some metrics:

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

### Residual Sum of Squares

$RSS = \sum_n (\hat{t}_n-t_n)^2$, it tells us how much of the prediction differs from the true value.

In [ ]:
RSS = np.sum((y_pred-y)**2)
RSS

### Coefficient of determination

$R^2 = 1 - \frac{RSS}{\sum_n (\bar{t}-t_n)^2}$, it tells us how the fraction of the variance of the data explained by the model (how much better we are doing w.r.t. just using the mean of the target $\bar{t} = \frac{\sum_n t_n}{N}$).

In spaces with a single feature this is equal to the correlation coefficient between the input and the output;

For a more detailed explanation: https://en.wikipedia.org/wiki/Coefficient_of_determination

In [ ]:
r2_score(y, y_pred)

### Mean Squared Error

$MSE = \frac{\sum_n (\hat{t}_n-t_n)^2}{N}$, it tells approximately how much error we get on a predicted data over the training set (i.e., a normalized version of the RSS).

In [ ]:
mean_squared_error(y, y_pred)

## Significance tests

Under the assumption that the observations $t_n$ are i.i.d. and satisfies $t_n = w_0 + \sum_j w_j x_{nj} + \epsilon$, where $\epsilon$ is a Gaussian noise with zero mean and variance $\sigma^2$ (i.e., the data are generated by a linear model with noise), the computed coefficients $\hat{w}_j$ are distributed as follows:
\begin{equation*}
	\frac{\hat{w}_j - w_j}{\hat{\sigma} \sqrt{v_j}} \sim t_{N - M -1}
\end{equation*}
where $w_j$ is the true parameter, $\hat{\sigma}$ is the unbiased estimated for the target variance, i.e., $\hat{\sigma}^2 = \frac{\sum_n (t_n - \hat{t}_n)^2}{N - M - 1}$, $v_j$ is the $j$-th diagonal element of the matrix $(X^T X)^{-1}$ and $t_{N - M-1}$ is the t-student distribution with $N - M - 1$ degrees of freedom.

This allow us to formulate some **statistical tests**:

### Single coefficients statistical test:
$$H_0: w_j = 0 \qquad \text{ vs. } \qquad H_1: w_j \neq 0$$
\begin{equation*}
t_{stat} = \frac{\hat{w}_j - w_j}{\hat{\sigma} \sqrt{v_j}} \sim t_{N - M - 1}
\end{equation*}
where $t_{N - M - 1}$ is the T-Student distribution with $N-M-1$ degrees of freedom

### Overall significance of the model: F-statistic

It considers the following hypothesis test:

$$H_0: w_1 = \dots = w_M = 0 \text{ vs. }  H_1: \exists w_j \neq 0$$


The F-statistic can be computed and is distributed as follows:
$$ F = \frac{dfe}{M }\frac{\sum_n (\overline{t}_n-t_n)^2- RSS}{RSS} \sim F_{M, N-M-1} $$

where $F_{M, N-M-1}$ is the Fisher-Snedecor distribution with parameters $M$ and $N-M-1$.

If one wants all the information about the output of a linear model in a single instruction, just use the library **statsmodels** and use the function **summary()** on the result of the Ordinary Least Square optimization procedure

In [ ]:
from statsmodels import api as sm
lin_model = sm.OLS(y, x).fit()
print(lin_model.summary())

In [ ]:
lin_model._results.params

In [ ]:
lin_model._results.k_constant

# **TODO:** Predicting Sepal Length

It was easy to predict petal width from petal length alone:

$$\hat{y} = w_0 + w_1 \cdot \mathrm{petal\_length}$$

Now you face a more challenging problem: predicting **sepal length** from the other variables (petal length, petal width, sepal width). There are many things you can do.

You can choose one among petal length, petal width, sepal width and re-use the solution above for single-input single-output linear regression.

You can use two or all of your input variables:

$$\hat{y} = w_0 + w_1 \cdot \mathrm{petal\_length} + w_2 \cdot \mathrm{petal\_width} + w_3 \cdot \mathrm{sepal\_width} = \sum_{i=0}^3w_i x_i = \mathbf{w}^\top \mathbf{x}$$

$\mathbf{x} = [1,\mathrm{petal\_length}, \mathrm{petal\_width}, \mathrm{sepal\_width}]^\top$.

You can consider even more complex models by introducing any set of **features** (arbitrary functions) of the input variables



Example:

* $\phi_1(\mathbf{x}) = \mathrm{petal\_length}$
* $\phi_2(\mathbf{x}) = \mathrm{petal\_length^2}$
* $\phi_3(\mathbf{x}) = \mathrm{petal\_width}\cdot \mathrm{sepal\_width}$
* ...

$$\hat{y} = w_0 + w_1 \cdot \phi_1(\mathbf{x}) + w_2 \cdot \phi_2(\mathbf{x}) + \dots + w_d\cdot\phi_d(\mathbf{x}) = \sum_{i=0}^dw_i \phi_i(\mathbf{x})  = \mathbf{w}^\top\mathbf{\phi}(\mathbf{x})$$


where $\mathbf{\phi}(\mathbf{x})= [1, \phi_1(\mathbf{x}), \phi_2(\mathbf{x}),\dots,\phi_d(\mathbf{x})]^\top$

In [ ]:
# Some useful snippets first
# How to extract multiple columns from a dataframe:
X = dataset[["petal-width", "sepal-width"]]
assert X.shape == (150, 2) # no need to add a "fake" second dimension this time
# How to normalize each column separately:
X = zscore(X, axis=1)
# How to stack numpy arrays along a new axis
x_1 = np.ones(4)
x_2 = np.zeros(4)
xx = np.stack((x_1, x_2), axis=1)
assert xx.shape == (4,2)
# How to concatenate numpy arrays along an existing axis
z_1 = np.ones((4, 1))
z_2 = np.zeros((4, 1))
zz = np.concatenate((z_1, z_2), axis=1)
assert zz.shape == (4, 2)

### WRITE YOUR CODE HERE ###


## Thought Experiment

The model that we train depends on the specific dataset that we have. The Iris dataset was collected by American botanist Edgar Anderson in the 1930s.  
How representative is it of the population of Iris flowers? Maybe Anderson was just lucky to find flowers following a particular law...

Like Anderson, we go into the wild and measure the petal length (x) and sepal length (y) of Iris flowers.

First we collect 50 flowers and set them apart for testing purposes.

Then, each day we collect a different dataset of 20 flowers and fit two models using linear regression to predict **sepal length from petal length**:

* A "simple" linear model where the only feature is x itself
* A "complex" polynomial model with $\mathbf{\phi} = [x, x^2]^\top$

In [ ]:
def sample_dataset(n=20):
  # Pretend this is the secret Nature's distribution of Iris flowers
  return dataset.sample(n)

In [ ]:
# Our test data
test_data = sample_dataset(n=50)
x_test = zscore(test_data['petal-length'].values)
y_test = zscore(test_data["sepal-length"].values)

plt.scatter(x_test, y_test)

In [ ]:
repetitions = 1000
m1 = linear_model.LinearRegression()
mse_1 = np.zeros(repetitions)

m2 = linear_model.LinearRegression()
mse_2 = np.zeros(repetitions)

def feature_map(x, d=2):
  return np.stack([x**i for i in range(1, d+1)], axis=1)

for i in range(repetitions):
  dat = sample_dataset()
  x = zscore(dat['petal-length'].values)
  y = zscore(dat["sepal-length"].values)

  m1.fit(x.reshape(-1, 1), y)
  y_1 = m1.predict(x_test.reshape(-1, 1))
  mse_1[i] = mean_squared_error(y_test, y_1)

  m2.fit(feature_map(x), y)
  y_2 = m2.predict(feature_map(x_test))
  mse_2[i] = mean_squared_error(y_test, y_2)

### Which model is better?

In [ ]:
print("M1: mean of MSEs=", np.mean(mse_1), ", std of MSEs=", np.std(mse_1))
print("M2: mean of MSEs=", np.mean(mse_2), ", std of MSEs=", np.std(mse_2))

In [ ]:
plt.figure(figsize=(16,9))

plt.scatter(zscore(dataset['petal-length'].values),
            zscore(dataset['sepal-length'].values),
            alpha=0.7)

for _ in range(20):
  dat = sample_dataset()
  x = zscore(dat['petal-length'].values)
  y = zscore(dat["sepal-length"].values)

  m1.fit(x.reshape(-1, 1), y)
  m2.fit(feature_map(x), y)

  #plt.scatter(x, y, label='true')

  w1 = m1.coef_
  w0 = m1.intercept_

  xx = np.linspace(min(x_test), max(x_test), 100)
  yy = m2.predict(feature_map(xx))
  plt.plot(xx, w0 + w1 * xx, "r--", alpha=0.3)
  plt.plot(xx, yy, color='orange', alpha=0.3)

plt.legend(prop={'size': 20})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.show()

In [ ]:
plt.boxplot((mse_1, mse_2));

### In practice we (typically) have just one dataset!
We will see how to choose the best model in following lectures...

# Homeworks

Here we propose some exercises in python for you. They are not mandatory, but they can be helpful to better understand the contents of the lecture, by giving you the opportunity to develop some code by yourself.

### 1) Custom Implementation

We can also implement Least-Squares from scratch, using its closed-form:

\begin{equation}
\hat{\mathbb{w}}_{OLS} = (\mathbb{\Phi}^{\top}\mathbb{\Phi})^{-1}\mathbb{\Phi}^{\top}\ \mathbb{t},
\end{equation}

where $\mathbb{\Phi}= (\phi(x_1), \dots, \phi(x_N))^{\top}$ and $\mathbb{t} = (t_1, \dots, t_N)^{\top}.$

By using **numpy**:


In [ ]:
from numpy.linalg import inv

### WRITE YOUR CODE HERE ###

### 2) Implementing LS for multiple outputs

We have seen at lesson that LS is possible also when we have multiple outputs.

Implement it by extending the LS custom implementation that we have seen.

In [ ]:
### WRITE YOUR CODE HERE ###

### 3) Try it on another dataset

Try to repeat the procedure that we have seen for the Iris dataset on a new dataset of your choice:

- select a dataset (many are available online, e.g. https://www.kaggle.com/datasets)
- visualize data, in order to spot interesting relationships
- preprocess data
- apply linear regression

In [ ]:
### WRITE YOUR CODE HERE ###